In [1]:
# imports

import pandas as pd

In [2]:
# read in data
# some of these lists are weird...and maybe shouldn't be used...
# or at least filtered by review and maybe genre to eliminate weird entries
fname = 'diverse_0_01_25.json'
data = pd.read_json(fname)
data

# The descriptions are screwed up - parts of them repeat.  I'm not sure what to do about this.  
# If first reducing to words, could live with it and use unique words.

,ISBN,author,avg_ratings,awards,character_names,description,genres,language,num_pages,num_ratings,num_reviews,places,publish_date,title,url
0,1607014017,Jacqueline Koyanagi,3.54,[James Tiptree Jr. Award Honor List (2014)],[],[Alana Quick is the best damned sky surgeon in...,"[Science Fiction, Lgbt, Glbt, Queer, Fiction, ...",English,331.0,"1,096",246,[],2013-12-04 00:00:00,Ascension,https://www.goodreads.com/book/show/17751274-a...
1,NaN,Valjeanne Jeffers,4.40,[],[],[AN IMMORTAL JOURNEY CONTINUES ... After a cen...,"[Speculative Fiction, Paranormal, Shapeshifter...",English,198.0,42,9,[],2010-11-27 00:00:00,The Time of Legend,https://www.goodreads.com/book/show/10465070-t...
2,0226971503,Wu Cheng'en,4.27,[],[],"[First published in 1592, , , volume I, compri...","[Classics, Cultural, China, Fiction, Fantasy, ...",English,544.0,933,69,[],1980-02-15 00:00:00,"The Journey to the West, Volume 1",https://www.goodreads.com/book/show/158788.The...
3,1933500263,Vandana Singh,3.89,[James Tiptree Jr. Award Honor List (2009)],[],"[ is fascinating far-future science fiction, s...","[Science Fiction, Science Fiction Fantasy, Sho...",English,154.0,38,5,[],2008-12-15 00:00:00,Distances: A Novella,https://www.goodreads.com/book/show/6271668-di...
4,0935312838,Rokeya Sakhawat Hossain,3.86,[],[],"[Sultana’s Dream, first published in 1905 in a...","[Fiction, Short Stories, Feminism, Cultural, I...",English,104.0,175,29,[],1993-01-01 00:00:00,Sultana's Dream: And Selections from The Seclu...,https://www.goodreads.com/book/show/910679.Sul...
5,1620409046,Raj Kamal Jha,3.29,[DSC Prize Nominee for South Asian Literature ...,[],"[As night falls in Delhi, a mother spins tales...","[Fiction, Cultural, India, Novels, Literary Fi...",English,352.0,154,43,[],2015-03-03 00:00:00,She Will Build Him a City,https://www.goodreads.com/book/show/22529401-s...
6,0143128795,Ahmed Saadawi,3.51,[الجائزة العالمية للرواية العربية (أي باف) / I...,"[محمود السوادي, الشسمه, هادي العتاگ, إيليشوا, ...","[""Brave and ingenious."" --, ""Gripping, darkly ...","[Fiction, Horror, Fantasy]",English,288.0,"6,365","1,306",[],2018-01-23 00:00:00,Frankenstein in Baghdad,https://www.goodreads.com/book/show/30780005-f...
7,4770016905,Kōbō Abe,3.68,[],[],[Abe's best short works to date are presented ...,"[Short Stories, Fiction, Asian Literature, Jap...",English,248.0,398,45,[],1993-02-01 00:00:00,Beyond the Curve,https://www.goodreads.com/book/show/26248.Beyo...
8,1600061230,Tosca Lee,4.17,"[Christy Award Nominee for First Novel (2008),...",[],[Clay has just been hired to write a memoir fo...,"[Christian Fiction, Fiction, Christian, Fantas...",English,336.0,"2,978",331,[],2007-05-18 00:00:00,Demon,https://www.goodreads.com/book/show/819789.Demon
9,081187107X,Sanjay Patel,4.56,[],[],[Artist and veteran Pixar animator Sanjay Pate...,"[Fantasy, Mythology, Cultural, India, Religion...",English,186.0,619,109,[],2010-02-10 00:00:00,Ramayana: Divine Loophole,https://www.goodreads.com/book/show/7456105-ra...


In [22]:
# clean summaries
data.loc[5,'description'][3:]

['As night falls in Delhi, a mother spins tales from her past for her sleeping daughter. Now grown up, her child is a puzzle with a million pieces, whom she hopes, through her words and her love, to somehow make whole again.',
 'Meanwhile, a young man rides the last train from Rajiv Chowk Station and dreams of murder.',
 'In another corner of the city, a newborn wrapped in a blood-red towel lies on the steps of an orphanage as his mother walks away.',
 'There are twenty million bodies in this city, but the stories of this woman, man, and child--of a secret love that blossoms in the shadows of grief, of a corrosive guilt that taints the soul, and of a boy who maps his own destiny--weave in and out of the lives of those around them to form a dazzling kaleidoscope of a novel.',
 'Beautiful, beguiling, and audacious, this is the story of a city and its people, of love and horror, of belonging and forgiveness: a powerful and unforgettable tale of modern India.']

In [19]:
data.loc[0,'description'][2:]

['Alana Quick is the best damned sky surgeon in Heliodor City, but repairing starship engines barely pays the bills. When the desperate crew of a cargo vessel stops by her shipyard looking for her spiritually advanced sister Nova, Alana stows away. Maybe her boldness will land her a long-term gig on the crew. But the ',
 " proves to be more than star-watching and plasma coils. The chief engineer thinks he's a wolf. The pilot fades in and out of existence. The captain is all blond hair, boots, and ego . . . and Alana can't keep her eyes off her. But there's little time for romance: Nova's in danger and someone will do anything--even destroying planets--to get their hands on her."]

In [20]:
data.loc[1,'description'][2:]

['AN IMMORTAL JOURNEY CONTINUES ... After a century Karla and Joseph are reunited and discover that they are werewolves - only to be marked for death by the sorcerer Tehotep. Escaping into the Time of Legend they find themselves trapped in a world of magic, blood and corruption. ',
 "The planet is at war. Homeless sleep in deserted buildings, while rich citizens live like kings. By night demonic gangs terrorize Topaz's streets. The lovers must find their kindred ... and start the revolution that will save Tundra. But are they too late?br/br/."]